In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sqlalchemy import create_engine
from crys import Crystal

import json

In [62]:
class CrysAlchemy(Crystal):
    def __init__(self, ip: str, dbname: str = 'meta', port: str = '5555', username: str = 'dbuser_dba'):
        self.ip = ip
        self.dbname = dbname
        self.port = port
        self.username = username
        self.crys_url = f"postgresql://{self.username}@{self.ip}:{self.port}/{self.dbname}"
        print(self.crys_url)
        
    def crys_conn(self):
        Crystal.__init__(self, ip=self.ip, conn_info=self.crys_url)
        self.cct_conn()

print("Please add your user: dbuser_dba password into ~/.pgpass file first")
input_meta_ip = input('Type gcp meta ip: ')
crys = CrysAlchemy(ip=input_meta_ip, dbname='meta', port='5555', username='dbuser_dba')


Please add your user: dbuser_dba password into ~/.pgpass file first
postgresql://dbuser_dba@35.221.148.74:5555/meta


In [63]:
crys.crys_conn()

['job',
 'default_var',
 'group',
 'host',
 'group_var',
 'host_var',
 'global_var',
 'twstock']

In [8]:
df = pd.read_sql_table('group_var', crys.crys_url, 'crystaldb').iloc[:, :3]
df

,cls,key,value
0,etcd,etcd_cluster,etcd
1,pg-meta,pg_cluster,pg-meta
2,pg-meta,pg_databases,"[{'name': 'meta', 'comment': 'crystaldb meta d..."
3,pg-meta,pg_users,"[{'name': 'dbuser_meta', 'roles': ['dbrole_adm..."
4,pg-meta,node_crontab,[00 01 * * * postgres /opt/CrystalDB/bin/pg-ba...
5,pg-lucas0,pg_cluster,pg-lucas0
6,pg-lucas0,pg_group,0
7,pg-lucas0,pg_mode,citus
8,pg-lucas0,pg_shard,pg-lucas
9,pg-lucas0,patroni_citus_db,lucas


In [10]:
df.loc[df['cls'] == 'pg-lucas0']

,cls,key,value
5,pg-lucas0,pg_cluster,pg-lucas0
6,pg-lucas0,pg_group,0
7,pg-lucas0,pg_mode,citus
8,pg-lucas0,pg_shard,pg-lucas
9,pg-lucas0,patroni_citus_db,lucas
10,pg-lucas0,pg_dbsu_password,DBUser.Postgres
11,pg-lucas0,pg_libs,"citus, timescaledb, pg_stat_statements, auto_e..."
12,pg-lucas0,pg_users,"[{'name': 'test', 'roles': ['dbrole_admin'], '..."
13,pg-lucas0,pg_databases,"[{'name': 'lucas', 'owner': 'citus', 'extensio..."
14,pg-lucas0,pg_hba_rules,"[{'db': 'all', 'addr': '10.140.15.0/24', 'auth..."


In [20]:
!gcloud compute instances list --format="json(name, networkInterfaces[0].networkIP, networkInterfaces[0].accessConfigs[0].natIP)" > gcp_instance.json

In [21]:
gcp_file_path = 'gcp_instance.json'

gcp_dat = json.load(open(gcp_file_path, 'r'))
print(gcp_dat)


[{'name': 'crystal-lab-0', 'networkInterfaces': [{'accessConfigs': [{'natIP': '34.81.153.103'}], 'networkIP': '10.140.15.218'}]}, {'name': 'crystal-lab-1', 'networkInterfaces': [{'accessConfigs': [{'natIP': '34.81.12.245'}], 'networkIP': '10.140.15.219'}]}, {'name': 'crystal-lab-2', 'networkInterfaces': [{'accessConfigs': [{'natIP': '104.199.154.44'}], 'networkIP': '10.140.15.220'}]}, {'name': 'joe-meta-1213', 'networkInterfaces': [{'accessConfigs': [{'natIP': '35.221.148.74'}], 'networkIP': '10.140.15.203'}]}, {'name': 'deb12', 'networkInterfaces': [{'accessConfigs': [{'natIP': '35.201.174.159'}], 'networkIP': '10.140.0.31'}]}, {'name': 'dev-8', 'networkInterfaces': [{'accessConfigs': [{'natIP': '34.81.208.253'}], 'networkIP': '10.140.0.25'}]}]


In [22]:
result_arr = [
    {
        "gcp_name": vm.get("name"),
        "internal_ip": vm.get("networkInterfaces")[0].get("networkIP"),
        "external_ip": vm.get("networkInterfaces")[0]
        .get("accessConfigs")[0]
        .get("natIP"),
    }
    for vm in gcp_dat
]

df_gcp = pd.DataFrame(result_arr)
df_gcp

,gcp_name,internal_ip,external_ip
0,crystal-lab-0,10.140.15.218,34.81.153.103
1,crystal-lab-1,10.140.15.219,34.81.12.245
2,crystal-lab-2,10.140.15.220,104.199.154.44
3,joe-meta-1213,10.140.15.203,35.221.148.74
4,deb12,10.140.0.31,35.201.174.159
5,dev-8,10.140.0.25,34.81.208.253


['crystal-lab-0',
 'crystal-lab-1',
 'crystal-lab-2',
 'joe-meta-1213',
 'deb12',
 'dev-8']

In [52]:
def get_all(con_url: str = "postgresql://dbuser_dba@35.221.148.74:5555/meta", schema: str = 'crystaldb'):
    cmdb_tbl_list = ['group', 'group_var', 'host', 'host_var']
    cmdb_tbl_col_len = [1, 3, 2, 4]
    for tbl, col_len in zip(cmdb_tbl_list, cmdb_tbl_col_len):
        df = pd.read_sql_table(tbl, con_url, schema).iloc[:, :col_len]
        print(df.shape)
        print(df.head())
        print("----"*22)
        
        
        
get_all()

(6, 1)
         cls
0        csi
1       etcd
2    pg-meta
3  pg-lucas0
4  pg-lucas1
----------------------------------------------------------------------------------------
(38, 3)
       cls           key                                              value
0     etcd  etcd_cluster                                               etcd
1  pg-meta    pg_cluster                                            pg-meta
2  pg-meta  pg_databases  [{'name': 'meta', 'comment': 'crystaldb meta d...
3  pg-meta      pg_users  [{'name': 'dbuser_meta', 'roles': ['dbrole_adm...
4  pg-meta  node_crontab  [00 01 * * * postgres /opt/CrystalDB/bin/pg-ba...
----------------------------------------------------------------------------------------
(6, 2)
         cls             ip
0        csi  10.140.15.203
1       etcd  10.140.15.203
2    pg-meta  10.140.15.203
3  pg-lucas0  10.140.15.218
4  pg-lucas1  10.140.15.219
----------------------------------------------------------------------------------------
(10, 4)
 

In [49]:
pd.read_sql_query('select * from crystaldb.shard_info', 'postgresql://dbuser_dba@35.221.148.74:5555/meta')

,shard,shard_seq,shard_role,cls,dbname,ip,host_role
0,pg-lucas,0,coordinator,pg-lucas0,lucas,10.140.15.218,primary
1,pg-lucas,1,worker,pg-lucas1,lucas,10.140.15.219,primary
2,pg-lucas,2,worker,pg-lucas2,lucas,10.140.15.220,primary


In [61]:
df_gv = pd.read_sql_table('group_var', 'postgresql://dbuser_dba@35.221.148.74:5555/meta', 'crystaldb').iloc[:, :3]
df_gv.loc[(df_gv['cls'] == 'pg-meta') & (df_gv['key'] == 'pg_users'), 'value'].iloc[0]

[{'name': 'dbuser_meta',
  'roles': ['dbrole_admin'],
  'comment': 'crystaldb admin user',
  'password': '{{ dbuser_meta_pwd }}',
  'pgbouncer': True},
 {'name': 'dbuser_view',
  'roles': ['dbrole_readonly'],
  'comment': 'read-only viewer for meta database',
  'password': '{{ dbuser_view_pwd }}',
  'pgbouncer': True}]

In [59]:
csv_file_path = '/Users/lucaslee/Documents/GitHub/ultraviolet/taiwan_stock_price_twse_2023-12-21.csv'

df_stock = pd.read_csv(csv_file_path)
df_stock.to_sql('twstock', crys.crys_url, 'crystaldb', if_exists='replace', index=False)

267

In [60]:
# check if the table is created
tbl_name = 'twstock'
df = pd.read_sql_table(tbl_name, crys.crys_url, 'crystaldb')
df.head()

,StockID,TradeVolume,Transaction,TradeValue,Open,Max,Min,Close,Change,date
0,0050,5619016,7953,743838778,132.20,132.70,132.05,132.60,-1.15,2023-12-21
1,0051,35903,183,2570060,71.60,71.85,71.20,71.75,-0.35,2023-12-21
2,0052,168529,283,21091275,124.80,125.75,124.70,125.75,-0.95,2023-12-21
3,0053,4871,1006,338250,69.30,69.90,69.30,69.80,-0.55,2023-12-21
4,0055,131797,233,3117477,23.69,23.70,23.60,23.70,-0.15,2023-12-21


In [3]:
pop = pd.read_csv('../data/2021_population.csv')
pop.head()

,iso_code,country,2021_last_updated,2020_population,area,density_sq_km,growth_rate,world_%,rank
0,CHN,China,"1,447,065,329","1,439,323,776","9,706,961 sq_km",149/sq_km,0.34%,18.34%,1
1,IND,India,"1,401,310,563","1,380,004,385","3,287,590 sq_km",424/sq_km,0.97%,17.69%,2
2,USA,United States,"334,058,426","331,002,651","9,372,610 sq_km",36/sq_km,0.58%,4.23%,3
3,IDN,Indonesia,"278,037,263","273,523,615","1,904,569 sq_km",145/sq_km,1.04%,3.51%,4
4,PAK,Pakistan,"227,724,796","220,892,340","881,912 sq_km",255/sq_km,1.95%,2.86%,5


In [11]:
pop['2021_last_updated'].str.replace(',', '').astype(int)

0      1447065329
1      1401310563
2       334058426
3       278037263
4       227724796
          ...    
219          4977
220          3533
221          1619
222          1373
223           800
Name: 2021_last_updated, Length: 224, dtype: int64

In [14]:
df = pd.read_sql_query('select * from crystaldb.shard_info', crys.crys_url)
df.head()

,shard,shard_seq,shard_role,cls,dbname,ip,host_role
0,pg-lucas,0,coordinator,pg-lucas0,lucas,10.140.15.218,primary
1,pg-lucas,1,worker,pg-lucas1,lucas,10.140.15.219,primary
2,pg-lucas,2,worker,pg-lucas2,lucas,10.140.15.220,primary


In [18]:
df[df['shard_role'] == 'coordinator']

,shard,shard_seq,shard_role,cls,dbname,ip,host_role
0,pg-lucas,0,coordinator,pg-lucas0,lucas,10.140.15.218,primary
